# Classification Data

In [ ]:
import flair #

from flair.data import Sentence
from flair.models import SequenceTagger
# load tagger
tagger = SequenceTagger.load("flair/ner-english-large") #加载了预训练NER模型
#model_name_or_path=r"D:\My Computer\py project\last\model\flair\pytorch_model.bin"
#tagger= SequenceTagger.load(model_name_or_path) ##本地模型

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)

In [ ]:
from tqdm import tqdm
import csv

In [ ]:
def replace_entities_placeholder_flair(text):
    # 用占位符来替换NER
    # make example sentence
    sentence = Sentence(text)
    # predict NER tags
    tagger.predict(sentence)
    # iterate over entities and print
    replacements = [] #存储待替换的实体信息
    if not sentence.get_spans('ner'):
        return text

    for entity in sentence.get_spans('ner'):
        #对于识别到的命名实体
        if entity.get_label().value == "ORG":
            repl = "ORG"
            replacements.append((entity.start_position, entity.end_position, repl, entity.text))
            #将替换信息（起始位置、结束位置、替换后的值、原始文本）存储到replacements列表中
        elif entity.get_label().value == "PER":
            repl = "PERSON"
            replacements.append((entity.start_position, entity.end_position, repl, entity.text))
        elif entity.get_label().value == "LOC":
            repl = "LOCATION"
            replacements.append((entity.start_position, entity.end_position, repl, entity.text))

    #完成替换，将识别替换后的文本重新组合成一个完整的字符串
    if replacements:
        res = []
        i = 0
        for (start, end, txt, orig) in replacements:
            assert orig != txt
            res.append(text[i:start] + txt)
            i = end
        res.append(text[end:])
        return ''.join(res)
    return text

In [ ]:
from datasets import load_dataset
cls_data = load_dataset("imdb")

Using the latest cached version of the dataset since imdb couldn't be found on the Hugging Face Hub


ValueError: Couldn't find cache for imdb for config 'default'
Available configs in the cache: ['plain_text']

In [7]:
cls_data

NameError: name 'cls_data' is not defined

In [5]:
train_data = cls_data['train']
unsup_data = cls_data['unsupervised']
test_data = cls_data['test']


NameError: name 'cls_data' is not defined

In [4]:
train_data[0]['text']

NameError: name 'train_data' is not defined

In [10]:
replace_entities_placeholder_flair(train_data[10]['text'].replace("<br /><br />", " ").replace("<br />", ""))
#预处理后传给替换函数

'It was great to see some of my favorite stars of 30 years ago including PERSON, PERSON and PERSON. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing. Some of the smaller female roles were fine, PERSON and PERSON were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think PERSON got a chance to act in this her only important film role. The film appears to have some fans, and I was very open-minded when I started watching it. I am a big PERSON fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one. It is ironic that this movie is about a detective agency where the detectives and clients get romantically involved with each 

In [19]:
train_pairs_placeholder = []
#匿名化ibdm的训练集
with open("data/flair/imdb_train.csv", "w",encoding='utf-8') as f:
    #打开文件
    writer = csv.writer(f)
    #打开csv
    writer.writerow(["text","label"])
    for p in tqdm(train_data):
        src = replace_entities_placeholder_flair(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        #对信息进行预处理
        train_pairs_placeholder.append((src, p['label']))
        writer.writerow((src, p['label']))

100%|██████████| 25000/25000 [9:34:09<00:00,  1.38s/it]   


In [11]:
test_pairs_placeholder = []
#匿名化测试集
with open("data/flair/imdb_test.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(test_data):
        src = replace_entities_placeholder_flair(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        test_pairs_placeholder.append((src, p['label']))
        writer.writerow((src, p['label']))

100%|██████████| 25000/25000 [1:23:24<00:00,  5.00it/s]  


In [12]:
unsup_pairs_placeholder = []
#匿名化未标记的数据集
with open("data/flair/imdb_unsup.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(unsup_data):
        src = replace_entities_placeholder_flair(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        unsup_pairs_placeholder.append((src, p['label']))
        writer.writerow((src, p['label']))

100%|██████████| 50000/50000 [2:53:05<00:00,  4.81it/s]  


# Spacy

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')#安装spacy

In [ ]:
def replace_entities_placeholder_spacy(text):
    parsed = nlp(text)
    # iterate over entities and print
    replacements = []
    if all([w.ent_type == 0 for w in parsed]):
        return text

    for word in parsed:
        if word.ent_type_ == "ORG":
            repl = "ORG"
            replacements.append((word.idx, word.idx + len(word.text), repl, word.text))
        elif word.ent_type_ == "PERSON":
            repl = "PERSON"
            replacements.append((word.idx, word.idx + len(word.text), repl, word.text))
        elif word.ent_type_ == "GPE":
            repl = "LOCATION"
            replacements.append((word.idx, word.idx + len(word.text), repl, word.text))

    if replacements:
        res = []
        i = 0
        for (start, end, txt, orig) in replacements:
            assert orig != txt
            res.append(text[i:start] + txt)
            i = end
        res.append(text[end:])
        return ''.join(res)
    return text

# IMDB

In [ ]:
train_pairs_placeholder2 = []
with open("data/spacy/imdb_train.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(train_data):
        src = replace_entities_placeholder_spacy(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        train_pairs_placeholder2.append((src, p['label']))
        writer.writerow((src, p['label']))

In [ ]:
test_pairs_placeholder2 = []
with open("data/spacy/imdb_test.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(test_data):
        src = replace_entities_placeholder_spacy(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        test_pairs_placeholder2.append((src, p['label']))
        writer.writerow((src, p['label']))

In [ ]:
unsup_pairs_placeholder2 = []
with open("data/spacy/imdb_unsup.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(unsup_data):
        src = replace_entities_placeholder_spacy(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        unsup_pairs_placeholder2.append((src, p['label']))
        writer.writerow((src, p['label']))